In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.0-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.5-py3-none-any.whl.metadata (9.5 kB)
  Using cached jiwer-4.0.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached gradio-5.42.0-py3-none-any.whl.metadata (16 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached torchcodec-0.6.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (

In [24]:
import torch
import librosa
from huggingface_hub import login
from dotenv import load_dotenv
import os
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load token dari .env
load_dotenv()
token = os.getenv("HF_TOKEN")
login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [25]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="train")
common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="validation")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="test")

print(common_voice)

Using the latest cached version of the dataset since mozilla-foundation/common_voice_17_0 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'id' at /home/lenovo/.cache/huggingface/datasets/mozilla-foundation___common_voice_17_0/id/17.0.0/9d10386a731ff6e6ed4ec973a4dc204a9820e8c842fbe388bdba0dd205ed5016 (last modified on Fri Aug 15 20:46:10 2025).
Using the latest cached version of the dataset since mozilla-foundation/common_voice_17_0 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'id' at /home/lenovo/.cache/huggingface/datasets/mozilla-foundation___common_voice_17_0/id/17.0.0/9d10386a731ff6e6ed4ec973a4dc204a9820e8c842fbe388bdba0dd205ed5016 (last modified on Fri Aug 15 20:46:10 2025).
Using the latest cached version of the dataset since mozilla-foundation/common_voice_17_0 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'id' at /home/lenovo/.cache/huggingface/datasets/m

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4970
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 3349
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 3641
    })
})


In [26]:
common_voice = common_voice.remove_columns(['client_id', 'path', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'])

In [27]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [28]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Indonesian", task="transcribe")

In [32]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6 and 7.
          2. The PyTorch version (2.8.0+cu128) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 7: libavutil.so.59: cannot open shared object file: No such file or directory
FFmpeg version 6: /lib/x86_64-linux-gnu/libpango-1.0.so.0: undefined symbol: g_once_init_leave_pointer
FFmpeg version 5: libavutil.so.57: cannot open shared object file: No such file or directory
FFmpeg version 4: libavutil.so.56: cannot open shared object file: No such file or 

In [14]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Indonesian", task="transcribe")

In [20]:
print(common_voice["train"].with_format("numpy"))

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 4970
})


In [18]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000)) # Cast sampling rate yang awalnya 48000 menjadi 16000

In [21]:
def prepare_dataset(batch):
    '''
    Fungsi ini melakukan praposes dataset, ingat bahwa ada dua komponen yang perlu dipraproses.
    - input_features adalah array audio (stereo) yang akan diproses oleh feature_extractor untuk menjadi spektogram.
    - labels adalah transkripsi yang akan diproses oleh tokenizer dengan menggunakan BPE untuk menjadi representasi numeriknya.
    '''
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch